# Description
This kernel creates an image dataset for train based on the competition data. Use of images allows to avoid loading the entire dataset into memory, which may be important for running experiments at kaggle. Meanwhile the inference can be done  by loading the dataset part by part without saving it as images to improve the speed.

The original images are cropped keeping only the characters and resized to 128x128 with adding the corresponding padding to maintain the aspect ratio (see images plot in the kernel). The stats of the produced images are also computed.

In [5]:
import cv2
from tqdm import tqdm_notebook as tqdm
import zipfile
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [6]:
HEIGHT = 137
WIDTH = 236
SIZE = 128

TRAIN = ['../input/bengaliai-cv19/train_image_data_0.parquet',
         '../input/bengaliai-cv19/train_image_data_1.parquet',
         '../input/bengaliai-cv19/train_image_data_2.parquet',
         '../input/bengaliai-cv19/train_image_data_3.parquet']

In [7]:
def bbox(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    return rmin, rmax, cmin, cmax

def crop_resize(img0, size=SIZE, pad=16):
    #crop a box around pixels large than the threshold 
    #some images contain line at the sides
    ymin,ymax,xmin,xmax = bbox(img0[5:-5,5:-5] > 80)
    #cropping may cut too much, so we need to add it back
    xmin = xmin - 13 if (xmin > 13) else 0
    ymin = ymin - 10 if (ymin > 10) else 0
    xmax = xmax + 13 if (xmax < WIDTH - 13) else WIDTH
    ymax = ymax + 10 if (ymax < HEIGHT - 10) else HEIGHT
    img = img0[ymin:ymax,xmin:xmax]
    #remove lo intensity pixels as noise
    img[img < 28] = 0
    lx, ly = xmax-xmin,ymax-ymin
    l = max(lx,ly) + pad
    #make sure that the aspect ratio is kept in rescaling
    img = np.pad(img, [((l-ly)//2,), ((l-lx)//2,)], mode='constant')
    return cv2.resize(img,(size,size))

In [13]:
df = pd.read_parquet(TRAIN[0])
n_imgs = 8
# fig, axs = plt.subplots(n_imgs, 2, figsize=(10, 5*n_imgs))

# # for idx in range(n_imgs):
# #     #somehow the original input is inverted
# #     img0 = 255 - df.iloc[idx, 1:].values.reshape(HEIGHT, WIDTH).astype(np.uint8)
# #     #normalize each image by its max val
# #     img = (img0*(255.0/img0.max())).astype(np.uint8)
# #     img = crop_resize(img)

# #     axs[idx,0].imshow(img0)
# #     axs[idx,0].set_title('Original image')
# #     axs[idx,0].axis('off')
# #     axs[idx,1].imshow(img)
# #     axs[idx,1].set_title('Crop & resize')
# #     axs[idx,1].axis('off')
# # plt.show()

In [14]:
x_tot,x2_tot = [],[]
imgs = []
with zipfile.ZipFile(OUT_TRAIN, 'w') as img_out:
    for fname in TRAIN:
        df = pd.read_parquet(fname)
        #the input is inverted
        data = 255 - df.iloc[:, 1:].values.reshape(-1, HEIGHT, WIDTH).astype(np.uint8)
        for idx in tqdm(range(len(df))):
            name = df.iloc[idx,0]
            #normalize each image by its max val
            img = (data[idx]*(255.0/data[idx].max())).astype(np.uint8)
            img = crop_resize(img)
        
#             x_tot.append((img/255.0).mean())
#             x2_tot.append(((img/255.0)**2).mean()) 
#             img = cv2.imencode('.png',img)[1]
#             img_out.writestr(name + '.png', img)
            imgs.append(img)

In [15]:
len(imgs)

200840

In [17]:
all_imgs = np.stack(imgs)

In [19]:
import pickle
with open('../input/bengaliai-cv19/train_images.pkl', 'wb') as f:
    pickle.dump(all_imgs, f)

In [10]:
#image stats
img_avr =  np.array(x_tot).mean()
img_std =  np.sqrt(np.array(x2_tot).mean() - img_avr**2)
print('mean:',img_avr, ', std:', img_std)

mean: 0.06922848809290576 , std: 0.20515700083327537


In [11]:
ls

EDA.ipynb*        image-preprocessing-128x128.ipynb  split_folds.py*
README.md*        losses/                            train.py*
Untitled.ipynb*   models/                            train.zip
Untitled1.ipynb*  optimizers/                        transforms/
build.py*         prediction.py*                     utils/
config/           results/                           validation.py*
data/             run.sh*                            wandb/
datasets/         schedulers/


In [ ]:
images = []
for p in 